In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/13 16:56:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [5]:
df_green.createOrReplaceTempView('green')

In [16]:
df_green_revenue = df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    count(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [25]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode="overwrite")

In [21]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.createOrReplaceTempView('yellow')

In [23]:
df_yellow_revenue = df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    
    SUM(total_amount) AS amount,
    count(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [26]:
df_yellow_revenue\
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode="overwrite")

In [2]:
df_green_revenue = spark.read.parquet('data/report/revenue/green/')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow/')

In [3]:
# Rename columns to diferentiate green from yellon on the join
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'green_yellow_records')

In [4]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [6]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [8]:
df_join = spark.read.parquet('data/report/revenue/total')

In [9]:
df_zones = spark.read.parquet('zones/')

In [12]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [14]:
df_result.drop('LocationID', 'zone').write.parquet('data/report/revenue/zones')

In [16]:
df_result.show()

+-------------------+----+------------------+--------------------+------------------+--------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|green_yellow_records|LocationID|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+--------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00|  14|              NULL|                NULL|               8.8|                   1|        14| Brooklyn|           Bay Ridge|   Boro Zone|
|2020-01-01 00:00:00|  15|              NULL|                NULL|             34.09|                   1|        15|   Queens|Bay Terrace/Fort ...|   Boro Zone|
|2020-01-01 00:00:00|  17|195.03000000000003|                   9|220.20999999999998|                   8|        17| Brooklyn|             Bedford|   Boro Zone|
|2020-01-01 00:00:00|  25| 5